In [1]:
%load_ext autoreload
%autoreload 2 
%reload_ext autoreload
%matplotlib inline

import matplotlib.pyplot as plt

import numpy as np
import scipy.io as io
from pyDOE import lhs
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

from complexPyTorch.complexLayers import ComplexLinear

import cplxmodule
from cplxmodule import cplx
from cplxmodule.nn import RealToCplx, CplxToReal, CplxSequential, CplxToCplx
from cplxmodule.nn import CplxLinear, CplxModReLU, CplxAdaptiveModReLU, CplxModulus, CplxAngle

# To access the contents of the parent dir
import sys; sys.path.insert(0, '../')
import os
from scipy.io import loadmat
from lightning_utils import *
from utils import *
from models import (TorchComplexMLP, ImaginaryDimensionAdder, cplx2tensor, 
                    ComplexTorchMLP, ComplexSymPyModule, complex_mse)
from models import UncertaintyWeightedLoss
from preprocess import *

# Model selection
from sparsereg.model import STRidge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge
from pde_diff import TrainSTRidge, FiniteDiff, print_pde
from robust_pde_diff import Robust_LRSTR, RobustPCA
from RegscorePy.bic import bic

from madgrad import MADGRAD

/usr/local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
# torch device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("You're running on", device)

DATA_PATH = '../PDE_FIND_experimental_datasets/harmonic_osc.mat'
data = io.loadmat(DATA_PATH)

t = data['t'].flatten()[:,None]
x = data['x'].flatten()[:,None]

spatial_dim = x.shape[0]
time_dim = t.shape[0]

potential = np.vstack([0.5*np.power(x,2).reshape((1, spatial_dim)) for _ in range(time_dim)])
Exact = data['usol']

X, T = np.meshgrid(x, t)

# Adjust the diemnsion of Exact and potential (0.5*x**2)
if Exact.T.shape == X.shape: Exact = Exact.T
if potential.T.shape == X.shape: potential = potential.T
Exact_u = np.real(Exact)
Exact_v = np.imag(Exact)

# Converting in a feature vector for each feature
X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
h_star = to_column_vector(Exact)
u_star = to_column_vector(Exact_u)
v_star = to_column_vector(Exact_v)
potential = to_column_vector(potential)

# Doman bounds
lb = X_star.min(axis=0)
ub = X_star.max(axis=0)

# Converting the grounds to be tensor
X_star = to_tensor(X_star, True)
h_star = to_complex_tensor(h_star, False)

N = 2000; include_N_res = 2
idx = np.random.choice(X_star.shape[0], N, replace=False)
# idx = np.arange(N) # Just have an easy dataset for experimenting

lb = to_tensor(lb, False).to(device)
ub = to_tensor(ub, False).to(device)

X_train = to_tensor(X_star[idx, :], True).to(device)
u_train = to_tensor(u_star[idx, :], False).to(device)
v_train = to_tensor(v_star[idx, :], False).to(device)
h_train = torch.complex(u_train, v_train).to(device)
potential = to_tensor(potential[idx, :], False).to(device)

# X_train = to_tensor(np.load("./tmp_files/X_train_2000labeledsamplesV3.npy")[:N, :], True).to(device)
# h_train = to_complex_tensor(np.load("./tmp_files/h_train_2000labeledsamplesV3.npy"), False).to(device)

# Unsup data -> We do not need this here.
# Potential is calculated from x
# Hence, Quadratic features of x are required.
feature_names = ['hf', 'h_xx', 'V']

You're running on cpu


/Users/pongpisit/Desktop/Multi-task-Physics-informed-neural-networks/inverse_qho/../utils.py:165: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(arr).float().requires_grad_(g)


In [3]:
# Define the initial coeffs
# u_t = (-0.000084 +0.494702i)h_xx
#     + (0.001434 -0.994890i)hf V

cn1 = (-0.000084-0.994890*1j)
cn2 = (0.001434+0.494702*1j)
cns = [cn1, cn2]

In [4]:
# Type the equation got from the symbolic regression step
# No need to save the eq save a pickle file before
program1 = "X0*X2"
pde_expr1, variables1,  = build_exp(program1); print(pde_expr1, variables1)

program2 = "X1"
pde_expr2, variables2,  = build_exp(program2); print(pde_expr2, variables2)

mod = ComplexSymPyModule(expressions=[pde_expr1, pde_expr2], complex_coeffs=cns, learnable_parts=[True, True]); mod.train()

X0*X2 {X2, X0}
X1 {X1}


ComplexSymPyModule(
  (sympymodule): SymPyModule(expressions=(X0*X2, X1))
)

In [5]:
class ComplexPINN(nn.Module):
    def __init__(self, model, loss_fn, index2features, scale=False, lb=None, ub=None, uncert=False):
        super(ComplexPINN, self).__init__()
        self.model = model
        self.callable_loss_fn = loss_fn
        self.index2features = index2features; self.feature2index = {}
        for idx, fn in enumerate(self.index2features): self.feature2index[fn] = str(idx)
        self.scale = scale; self.lb, self.ub = lb, ub
        if self.scale and (self.lb is None or self.ub is None): 
            print("Please provide thw lower and upper bounds of your PDE.")
            print("Otherwise, there will be error(s)")
        self.diff_flag = diff_flag(self.index2features)
        self.uncert = None
        if uncert: self.uncert = UncertaintyWeightedLoss(2)
        
    def forward(self, x, t):
        H = torch.cat([x, t], dim=1)
        if self.scale: H = self.neural_net_scale(H)
        return self.model(H)
    
    def loss(self, x, t, potential, y_input, update_network_params=True, update_pde_params=True):
        total_loss = []
        grads_dict, u_t = self.grads_dict(x, t, potential)
        # MSE Loss
        if update_network_params:
            total_loss.append(complex_mse(grads_dict['X0'], y_input))
        # PDE Loss
        if update_pde_params:
            total_loss.append(complex_mse(self.callable_loss_fn(grads_dict), u_t))
            
        if self.uncert is not None: return self.uncert(*total_loss)
        else: return total_loss
    
    def grads_dict(self, x, t, potential):
        uf = self.forward(x, t)
        u_t = complex_diff(uf, t)
        
        ### PDE Loss calculation ###
        derivatives = {}
        derivatives['X0'] = cplx2tensor(uf)
        derivatives['X1'] = complex_diff(complex_diff(uf, x), x)
        derivatives['X2'] = potential
        
        return derivatives, u_t
    
    def gradients(self, func, x):
        return grad(func, x, create_graph=True, retain_graph=True, grad_outputs=torch.ones(func.shape))
    
    # Must ensure that the implementation of neural_net_scale is consistent
    # and hopefully correct
    # also, you might not need this function in some datasets
    def neural_net_scale(self, inp): 
        return 2*(inp-self.lb)/(self.ub-self.lb)-1

In [6]:
inp_dimension = 2
act = CplxToCplx[torch.tanh]
complex_model = CplxSequential(
                            CplxLinear(100, 100, bias=True),
                            act(),
                            CplxLinear(100, 100, bias=True),
                            act(),
                            CplxLinear(100, 100, bias=True),
                            act(),
                            CplxLinear(100, 100, bias=True),
                            act(),
                            CplxLinear(100, 1, bias=True),
                            )

complex_model = torch.nn.Sequential(
                                    torch.nn.Linear(inp_dimension, 200),
                                    RealToCplx(),
                                    complex_model
                                    )

/usr/local/lib/python3.9/site-packages/torch/nn/modules/container.py:587: UserWarning: Setting attributes on ParameterDict is not supported.
  warnings.warn("Setting attributes on ParameterDict is not supported.")


In [7]:
# Pretrained model
semisup_model_state_dict = cpu_load("./saved_path_inverse_qho/qho_complex_model_2000labeledsamples_jointtrainwith4000unlabeledsamplesV3.pth")
parameters = OrderedDict()

# Filter only the parts that I care about renaming (to be similar to what defined in TorchMLP).
inner_part = "network.model."
for p in semisup_model_state_dict:
    if inner_part in p:
        parameters[p.replace(inner_part, "")] = semisup_model_state_dict[p]
complex_model.load_state_dict(parameters)

pinn = ComplexPINN(model=complex_model, loss_fn=mod, index2features=feature_names, scale=True, lb=lb, ub=ub, uncert=False)
pinn = load_weights(pinn, "tmp.pth")

Loaded the model's weights properly


In [8]:
def closure():
    global X_train, h_train, potential
    if torch.is_grad_enabled(): optimizer2.zero_grad(set_to_none=True)
    losses = pinn.loss(X_train[:, 0:1], X_train[:, 1:2], potential, h_train, update_network_params=True, update_pde_params=True)
    l = sum(losses)
    if l.requires_grad: l.backward(retain_graph=True)
    return l

def mtl_closure():
    global X_train, h_train, potential
    n_obj = 2 # There are two tasks
    losses = pinn.loss(X_train[:, 0:1], X_train[:, 1:2], potential, h_train, update_network_params=True, update_pde_params=True)
    updated_grads = []
    
    for i in range(n_obj):
        optimizer1.zero_grad(set_to_none=True)
        losses[i].backward(retain_graph=True)

        g_task = []
        for param in pinn.parameters():
            if param.grad is not None:
                g_task.append(Variable(param.grad.clone(), requires_grad=False))
            else:
                g_task.append(Variable(torch.zeros(param.shape), requires_grad=False))
        # appending the gradients from each task
        updated_grads.append(g_task)

    updated_grads = list(pcgrad.pc_grad_update(updated_grads))[0]
    for idx, param in enumerate(pinn.parameters()): 
        param.grad = (updated_grads[0][idx]+updated_grads[1][idx])
        
    return sum(losses)

In [9]:
p1, p2 = see_params(pinn.callable_loss_fn)
torch.complex(p1, p2)

tensor([[-0.0083-0.9968j],
        [ 0.0088+0.4921j]])

In [10]:
errs = abs(np.array([100j*(1-0.9968), 100j*(0.5-0.4950)/0.5]))
np.mean(errs), np.std(errs) # 2.05 +- 0.39

(0.6599999999999995, 0.3400000000000014)

In [11]:
n_test = X_star.shape[0]
n_test = spatial_dim*np.ceil(n_test/spatial_dim)
test_idx = np.arange(n_test)
# test_idx = np.random.choice(X_star.shape[0], n_test, replace=False)

In [12]:
xx, tt = dimension_slicing(X_star[test_idx, :])

In [13]:
gd, h_t = pinn.grads_dict(xx, tt, 0.5*(xx**2)[test_idx, :])

In [14]:
referenced_derivatives = cat(*list(gd.values()))

In [15]:
derivatives = referenced_derivatives.detach().numpy()

dictionary = {}
for i in range(len(feature_names)): dictionary[feature_names[i]] = get_feature(derivatives, i)

c_poly = ComplexPolynomialFeatures(feature_names, dictionary)
complex_poly_features = c_poly.fit()

Computing hf
Computing h_xx
Computing V
Computing hf^2
Computing hf h_xx
Computing hf V
Computing h_xx^2
Computing h_xx V
Computing V^2


In [16]:
pred_Exact = referenced_derivatives[:, 0:1].reshape((int(spatial_dim), int(n_test//spatial_dim))).detach()
Z, E1 = RobustPCA(pred_Exact, lam_2=0.001)
complex_poly_features[:, 0:1] = to_column_vector(Z)

Please ensure that the shape of U is correct.
iteration:1, err:25034.412222887466, nc_norm:894.0360098638092 eta1:0.007047486302441743
iteration:50, err:101.77410269173963, nc_norm:1292.7806066028145 eta1:0.7521003888196454
iteration:100, err:0.8916539777836258, nc_norm:689.4859967031524 eta1:88.28970609468897
iteration:150, err:0.003867425433608397, nc_norm:684.3352549807297 eta1:10364.403898953218
iteration:200, err:2.0426641840410834e-05, nc_norm:684.2898544829554 eta1:1216686.213287763
iteration:207, err:9.907857375522878e-06, nc_norm:684.289663161962 eta1:2370977.2291681124


In [17]:
Ut1 = np.reshape(to_numpy(h_t), (int(spatial_dim), int(n_test//spatial_dim)))
w, X, E2 = Robust_LRSTR(complex_poly_features, Ut1, c_poly.poly_feature_names, lam_1=1e-5, lam_3=0.3, lam_4=1e-4, d_tol=500)

iteration:1, err:5435.652231767041, nc_norm:2583.8183532097005 eta2:0.004379923816486254
u_t = 
u_t = (0.002150 +0.496235i)h_xx
    + (0.000024 -0.998133i)hf V
   
u_t = (0.002135 +0.497022i)h_xx
    + (-0.000010 -0.999669i)hf V
   
u_t = (0.002126 +0.496620i)h_xx
    + (-0.000023 -0.998932i)hf V
   
u_t = (0.002124 +0.496557i)h_xx
    + (-0.000028 -0.998915i)hf V
   
u_t = (0.002124 +0.496505i)h_xx
    + (-0.000030 -0.998911i)hf V
   
u_t = (0.002125 +0.496469i)h_xx
    + (-0.000031 -0.998911i)hf V
   
u_t = (0.002125 +0.496446i)h_xx
    + (-0.000031 -0.998912i)hf V
   
u_t = (0.002126 +0.496433i)h_xx
    + (-0.000032 -0.998913i)hf V
   
u_t = (0.002126 +0.496425i)h_xx
    + (-0.000033 -0.998914i)hf V
   
u_t = (0.002126 +0.496420i)h_xx
    + (-0.000033 -0.998915i)hf V
   


KeyboardInterrupt: 

In [25]:
derivatives = referenced_derivatives.detach().numpy()

dictionary = {}
for i in range(len(feature_names)): dictionary[feature_names[i]] = get_feature(derivatives, i)

c_poly = ComplexPolynomialFeatures(feature_names, dictionary)
complex_poly_features = c_poly.fit()

w = TrainSTRidge(complex_poly_features, to_numpy(h_t), 1e-10, d_tol=1000, maxit=1000, l0_penalty=5, normalize=1)
print("PDE derived using STRidge")
print_pde(w, c_poly.poly_feature_names)

Computing hf
Computing h_xx
Computing V
Computing hf^2
Computing hf h_xx
Computing hf V
Computing h_xx^2
Computing h_xx V
Computing V^2
PDE derived using STRidge
u_t = (0.002135 +0.496425i)h_xx
    + (0.000018 -0.998858i)hf V
   


In [ ]:
# u_t = (0.002135 +0.497022i)h_xx
#     + (-0.000010 -0.999669i)hf V

# u_t = (0.002135 +0.497022i)h_xx
#     + (-0.000010 -0.999669i)hf V

In [ ]:
# 0.9868, 0.4911
# 1.0230, 0.4948
# 0.9968, 0.4950
# 0.9989, 0.4970
# 0.9997, 0.4971 # use this...